# Make pipe, then GridSearch
Run through the night.... uh turns out you didn't really need to.

Gridsearch on both ready.dat and reduce.dat. Can?

In [1]:
from dataload import *

In [2]:
x_train, x_test, y_train, y_test = load_object('reduce.dat')

In [3]:
x,y = load_object("samplexy.dat")

In [4]:
from sklearn import svm,model_selection
parameters = [
    {'kernel': ['linear'], 'C':[0.1, 1, 10]},
    {'kernel': ['rbf'], 'gamma':[0.5, 1, 2], 'C':[0.1, 1, 10]}]

In [5]:
clf = model_selection.GridSearchCV(svm.SVC(), parameters, 
                                   cv = model_selection.StratifiedKFold(n_splits = 3, 
                                                                        shuffle = True, 
                                                                        random_state = 2018),n_jobs=5)

In [6]:
%time clf.fit(x,y)

Wall time: 9min 30s


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=2018, shuffle=True),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=5,
       param_grid=[{'kernel': ['linear'], 'C': [0.1, 1, 10]}, {'kernel': ['rbf'], 'gamma': [0.5, 1, 2], 'C': [0.1, 1, 10]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [7]:
print('best score:', clf.best_score_)
print('best parameters: ', clf.best_params_)

best score: 0.9871929064302143
best parameters:  {'C': 10, 'gamma': 2, 'kernel': 'rbf'}


This is modified from ```models.py```.

I want y to point directly at the ```attack_type``` column.

In [18]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,cohen_kappa_score,roc_auc_score
def score_test(model, x,y):
    from pandas import DataFrame
    y_pred = model.predict(x)
    labels = ['normal', 'dos', 'probe', 'r2l', 'u2r']
    return DataFrame({
        'precision' : precision_score(y, y_pred, average=None, labels=labels),
        'recall' : recall_score(y, y_pred, average=None, labels=labels),
        'F1' : f1_score(y, y_pred, average=None, labels=labels),
        'kappa' : cohen_kappa_score(y, y_pred, labels=labels) },
        # 'AUC' : roc_auc_score(y_test.attack_type, y_pred, average=None) },
        index=labels)

In [15]:
y_test_atk = y_test.attack_type # sometimes this throws an error otherwise.

In [19]:
score_test(clf,x_test,y_test_atk)

,F1,kappa,precision,recall
normal,0.994759,0.993202,0.997344,0.992187
dos,0.999550,0.993202,0.999979,0.999122
probe,0.971963,0.993202,0.945890,0.999513
r2l,0.270000,0.993202,0.157205,0.955752
u2r,0.256410,0.993202,0.147059,1.000000


In [21]:
xt, xe, yt, ye = load_object('ready.dat')
ye = ye.attack_type

In [23]:
from sampler import make_pipe
samp_pipe = make_pipe(29999, levels=['dos', 'normal', 'probe'])
xtr, ytr = samp_pipe.fit_sample(xt, yt.attack_type)

In [24]:
save_object([xtr,ytr],"samplexy.dat")

In [25]:
%time clf.fit(xtr,ytr)

Wall time: 11min 18s


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=2018, shuffle=True),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=5,
       param_grid=[{'kernel': ['linear'], 'C': [0.1, 1, 10]}, {'kernel': ['rbf'], 'gamma': [0.5, 1, 2], 'C': [0.1, 1, 10]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [26]:
print('best score:', clf.best_score_)
print('best parameters: ', clf.best_params_)

best score: 0.9870862362078736
best parameters:  {'C': 10, 'gamma': 2, 'kernel': 'rbf'}


In [27]:
%time score_test(clf,xtr,ytr)

Wall time: 48.3 s


,F1,kappa,precision,recall
normal,0.997094,0.987375,0.999163,0.995033
dos,0.999733,0.987375,0.999967,0.999500
probe,0.999350,0.987375,0.999200,0.999500
r2l,0.975747,0.987375,0.976497,0.974999
u2r,0.977615,0.987375,0.974780,0.980466


In [28]:
%time score_test(clf,xe,ye)

Wall time: 2min 38s


,F1,kappa,precision,recall
normal,0.995561,0.994139,0.997369,0.993760
dos,0.999552,0.994139,0.999972,0.999132
probe,0.969776,0.994139,0.941972,0.999270
r2l,0.331230,0.994139,0.201536,0.929204
u2r,0.333333,0.994139,0.200000,1.000000
